In [ ]:
import geemap
import ee
from Monarch import user_gee

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = geemap.geojson_to_ee(r'H:/Monarch/paper/Data/area.geojson').geometry()
Map.center_object(roi, 12)
Map.addLayer(roi, {}, 'roi')

In [ ]:
visParams = {'max': 3000, 'min': 0, 'bands': ['B4', 'B3', 'B2']}

In [ ]:
for year in range(2018, 2019):
    s2_year = []
    for i in range(11, 12):
        s2 = ee.ImageCollection("COPERNICUS/S2")\
                .filterBounds(roi)\
                .sort('CLOUD_COVERAGE_ASSESSMENT', False)\
                .filterDate(str(year), str(year+1))\
                .filter(ee.Filter.calendarRange(i, i, 'month'))\
                .map(user_gee.rm_cloud_s2_sr)\
                .mosaic().clip(roi)
        Map.addLayer(s2, visParams, f"s2_{i}")
        s2_year.append(s2)
    s2_year = ee.ImageCollection(s2_year)

In [ ]:
year = 2018
modis = ee.ImageCollection("MODIS/006/MOD09A1").filterDate(str(year), str(year+1))\
        .map(lambda img: img.clip(roi))
ndvi = modis.map(lambda img: img.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']))
lswi = modis.map(lambda img: img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']))

In [ ]:
ndvi_